## Importing required libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

## Opening the 'Churn_Modelling.csv' file and creating a DataFrame

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Dropping the 'RowNumber', 'CustomerId' and 'Surname' columns because they are not useful for the model

In [3]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

## Encode the categorical columns

### Encoding the categorical column 'Gender' using LabelEncoder

In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


### One hot encoding the categorical variable 'Geography'

In [5]:
# Create a OneHotEncoder object, and fit it to geography column
oh_encoder_geography = OneHotEncoder(sparse_output=False, dtype=np.int64)
geography_ohe = oh_encoder_geography.fit_transform(data[['Geography']])

# Add the new columns to the dataframe
data = pd.concat([data, pd.DataFrame(geography_ohe, columns=oh_encoder_geography.get_feature_names_out())], axis=1)

# Drop the original column
data = data.drop('Geography', axis=1)

In [6]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


## Save the encoders

In [7]:
with open("label_encoder_gender.pkl", "wb") as f:
    pickle.dump(label_encoder_gender, f)

with open("oh_encoder_geography.pkl", "wb") as f:
    pickle.dump(oh_encoder_geography, f)

## Split the data into features and target

In [8]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

## ANN Regression Model

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## Build the model

In [12]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1
    Dense(32, activation='relu'), # HL2
    Dense(1) # OL
])

c:\Users\Ashutosh Choudhari\.conda\envs\annrnn\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Compile the model

In [13]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.MeanAbsoluteError()

In [14]:
model.compile(optimizer=opt, loss=loss, metrics=['mae'])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

## Set up TensorBoard

In [16]:
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

## Set up EarlyStopping

In [17]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights= True)

## Train the model

In [18]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorflow_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 96469.7500 - mae: 96469.7500 - val_loss: 53131.3672 - val_mae: 53131.3672
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51328.0430 - mae: 51328.0430 - val_loss: 50423.9023 - val_mae: 50423.9023
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 50605.6445 - mae: 50605.6445 - val_loss: 50241.4258 - val_mae: 50241.4258
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49987.3906 - mae: 49987.3906 - val_loss: 50203.2852 - val_mae: 50203.2852
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49871.3047 - mae: 49871.3047 - val_loss: 50197.9883 - val_mae: 50197.9883
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49441.1719 - mae: 49441.1719 - val_loss: 50144.0000 - val_mae: 50144.0000
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49803.9844 - mae: 49803.9844 - val_loss: 50241.3633 - val_mae: 50241.3633
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - 

## Save the model

In [19]:
model.save('regression_model.h5')

## Load tensorboard extension

In [20]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 35792), started 0:00:03 ago. (Use '!kill 35792' to kill it.)

## Evaluate the model on the test data

In [22]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 50991.9844 - mae: 50991.9844
Test loss: 50144.0
